# Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat.

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [2]:
#Reading the data file
checkins = pd.read_csv('checkins.dat', sep='|', skipinitialspace=True, skiprows=[1], low_memory=False)
print(checkins.info())
checkins.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021967 entries, 0 to 1021966
Data columns (total 6 columns):
id                  1021967 non-null object
user_id             1021966 non-null float64
venue_id            1021966 non-null float64
latitude            396634 non-null float64
longitude           396634 non-null float64
created_at          1021966 non-null object
dtypes: float64(4), object(2)
memory usage: 46.8+ MB
None


,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
2,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
4,984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58


In [3]:
#Stripping columns names (deleting whitespaces)
checkins.columns = checkins.columns.str.strip()

#Dropping rows with NaN values
checkins = checkins.dropna()
print('Shape without NaN: %s' % str(checkins.shape))
checkins.head()

Shape without NaN: (396634, 6)


,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
7,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
9,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [4]:
#Array for clustering (100000 entries and 'latitude' and 'longitude' columns)
checkins_cl = checkins.iloc[:100000, :].loc[:, ['latitude', 'longitude']]
print('Shape: ' + str(checkins_cl.shape))
checkins_cl.head()

Shape: (100000, 2)


,latitude,longitude
1,38.895112,-77.036366
3,33.800745,-84.410520
7,45.523452,-122.676207
9,40.764462,-111.904565
10,33.448377,-112.074037


In [5]:
#Clustering with MeanShift
ms = MeanShift(bandwidth=0.1)
ms.fit(checkins_cl)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [6]:
#Labels and cluster centers
labels = ms.labels_
cluster_centers = ms.cluster_centers_

print('Number of estimated clusters : %d' % len(cluster_centers))

#Count the number of items for each cluster
labels_unique, labels_counts = np.unique(labels, return_counts=True)
for lbl, cnt in zip(labels_unique, labels_counts):
    print(lbl, cnt)

Number of estimated clusters : 3230
(0, 12506)
(1, 4692)
(2, 3994)
(3, 3363)
(4, 3526)
(5, 2409)
(6, 2297)
(7, 1601)
(8, 1526)
(9, 1378)
(10, 1298)
(11, 1081)
(12, 1006)
(13, 1007)
(14, 714)
(15, 868)
(16, 870)
(17, 645)
(18, 808)
(19, 807)
(20, 612)
(21, 722)
(22, 754)
(23, 747)
(24, 539)
(25, 656)
(26, 580)
(27, 577)
(28, 679)
(29, 564)
(30, 594)
(31, 907)
(32, 449)
(33, 502)
(34, 452)
(35, 104)
(36, 431)
(37, 410)
(38, 388)
(39, 400)
(40, 369)
(41, 367)
(42, 384)
(43, 347)
(44, 345)
(45, 342)
(46, 314)
(47, 273)
(48, 314)
(49, 316)
(50, 355)
(51, 281)
(52, 336)
(53, 293)
(54, 271)
(55, 246)
(56, 263)
(57, 258)
(58, 254)
(59, 243)
(60, 229)
(61, 291)
(62, 182)
(63, 155)
(64, 137)
(65, 193)
(66, 186)
(67, 197)
(68, 196)
(69, 189)
(70, 191)
(71, 187)
(72, 203)
(73, 178)
(74, 192)
(75, 169)
(76, 173)
(77, 153)
(78, 157)
(79, 199)
(80, 220)
(81, 164)
(82, 162)
(83, 155)
(84, 126)
(85, 152)
(86, 56)
(87, 100)
(88, 141)
(89, 142)
(90, 190)
(91, 117)
(92, 138)
(93, 133)
(94, 135)
(95, 131)


(940, 7)
(941, 6)
(942, 7)
(943, 7)
(944, 7)
(945, 7)
(946, 7)
(947, 7)
(948, 7)
(949, 7)
(950, 7)
(951, 7)
(952, 10)
(953, 7)
(954, 7)
(955, 7)
(956, 7)
(957, 8)
(958, 7)
(959, 7)
(960, 7)
(961, 7)
(962, 7)
(963, 7)
(964, 7)
(965, 7)
(966, 7)
(967, 7)
(968, 7)
(969, 7)
(970, 7)
(971, 7)
(972, 7)
(973, 7)
(974, 7)
(975, 7)
(976, 7)
(977, 7)
(978, 7)
(979, 7)
(980, 7)
(981, 7)
(982, 6)
(983, 7)
(984, 7)
(985, 7)
(986, 7)
(987, 7)
(988, 7)
(989, 7)
(990, 7)
(991, 7)
(992, 7)
(993, 7)
(994, 7)
(995, 7)
(996, 7)
(997, 7)
(998, 7)
(999, 7)
(1000, 6)
(1001, 6)
(1002, 6)
(1003, 6)
(1004, 6)
(1005, 6)
(1006, 6)
(1007, 6)
(1008, 6)
(1009, 6)
(1010, 6)
(1011, 6)
(1012, 6)
(1013, 6)
(1014, 6)
(1015, 6)
(1016, 6)
(1017, 6)
(1018, 7)
(1019, 6)
(1020, 6)
(1021, 6)
(1022, 6)
(1023, 6)
(1024, 6)
(1025, 6)
(1026, 6)
(1027, 6)
(1028, 6)
(1029, 6)
(1030, 6)
(1031, 6)
(1032, 6)
(1033, 6)
(1034, 5)
(1035, 6)
(1036, 6)
(1037, 7)
(1038, 6)
(1039, 6)
(1040, 6)
(1041, 6)
(1042, 6)
(1043, 6)
(1044, 6)
(1045, 6)

(2504, 1)
(2505, 1)
(2506, 1)
(2507, 1)
(2508, 1)
(2509, 1)
(2510, 1)
(2511, 1)
(2512, 1)
(2513, 1)
(2514, 1)
(2515, 1)
(2516, 1)
(2517, 1)
(2518, 1)
(2519, 1)
(2520, 1)
(2521, 1)
(2522, 1)
(2523, 1)
(2524, 1)
(2525, 1)
(2526, 1)
(2527, 1)
(2528, 1)
(2529, 1)
(2530, 1)
(2531, 1)
(2532, 1)
(2533, 1)
(2534, 1)
(2535, 1)
(2536, 1)
(2537, 1)
(2538, 1)
(2539, 1)
(2540, 1)
(2541, 1)
(2542, 1)
(2543, 1)
(2544, 1)
(2545, 1)
(2546, 1)
(2547, 1)
(2548, 1)
(2549, 1)
(2550, 1)
(2551, 1)
(2552, 1)
(2553, 1)
(2554, 1)
(2555, 1)
(2556, 1)
(2557, 1)
(2558, 1)
(2559, 1)
(2560, 1)
(2561, 1)
(2562, 1)
(2563, 1)
(2564, 1)
(2565, 1)
(2566, 1)
(2567, 1)
(2568, 1)
(2569, 1)
(2570, 1)
(2571, 1)
(2572, 1)
(2573, 1)
(2574, 1)
(2575, 1)
(2576, 1)
(2577, 1)
(2578, 1)
(2579, 1)
(2580, 1)
(2581, 1)
(2582, 1)
(2583, 1)
(2584, 1)
(2585, 1)
(2586, 1)
(2587, 1)
(2588, 1)
(2589, 1)
(2590, 1)
(2591, 1)
(2592, 1)
(2593, 1)
(2594, 1)
(2595, 1)
(2596, 1)
(2597, 1)
(2598, 1)
(2599, 1)
(2600, 1)
(2601, 1)
(2602, 1)
(2603, 1)


In [7]:
#Determining the labels with counts more than 15
labels_unique_more = []
for index, cnt in enumerate(labels_counts):
    if cnt > 15:
        labels_unique_more.append(labels_unique[index])

print('Number of labels with counts more than 15: %d' % len(labels_unique_more))

Number of labels with counts more than 15: 593


In [8]:
#Determining cluster centers for the labels with counts more than 15
cluster_centers_more = np.empty( (len(labels_unique_more), 2) )
for index, lbl in enumerate(labels_unique_more):
    cluster_centers_more[index] = cluster_centers[lbl]

print(cluster_centers_more)

[[  40.7177164   -73.99183542]
 [  33.44943805 -112.00213969]
 [  33.44638027 -111.90188756]
 ..., 
 [  41.61853175  -88.44556818]
 [  38.65877915  -76.8856871 ]
 [  39.2494686   -77.1821271 ]]


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [9]:
#Reading the Carnival Cruise Line offices file
offices = pd.read_csv('carnival_cruise_lines_officies.txt', skipinitialspace=True)
print(offices.info())
offices_for_dist = offices.loc[:, ['latitude', 'longitude']]
offices_for_dist

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
latitude     6 non-null float64
longitude    6 non-null float64
city         6 non-null object
dtypes: float64(2), object(1)
memory usage: 216.0+ bytes
None


,latitude,longitude
0,33.751277,-118.188740
1,25.867736,-80.324116
2,51.503016,-0.075479
3,52.378894,4.885084
4,39.366487,117.036146
5,-33.868457,151.205134


In [10]:
def distance(x, y):
    return np.sqrt( np.sum((x - y)**2) )

In [11]:
#Calculating distances from nearest offices
distances_from_nearest_office = np.empty( cluster_centers_more.shape[0] )
for i, item in enumerate(cluster_centers_more):
    min_dist = distance(item, offices_for_dist.loc[0])
    for j in range(len(offices)):
        dist = distance(item, offices_for_dist.loc[j])
        if dist < min_dist:
            min_dist = dist
    distances_from_nearest_office[i] = min_dist

In [12]:
#Sorting arrays on minimum distance
dist_sorted_indices = np.argsort(distances_from_nearest_office)
distances_from_nearest_office_sorted = distances_from_nearest_office[dist_sorted_indices]
cluster_centers_more_sorted = cluster_centers_more[dist_sorted_indices]

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [13]:
print('Minimal distance: %f' % distances_from_nearest_office_sorted[0])
print('Coordinates of closest centroid: %s' % str(cluster_centers_more_sorted[0]))

Minimal distance: 0.007835
Coordinates of closest centroid: [ -33.86063043  151.20477593]


In [14]:
#Writing the answer to the file
answer = str(cluster_centers_more_sorted[0, 0]) + ' ' + str(cluster_centers_more_sorted[0, 1])
with open('answer.txt', 'w') as fout:
    fout.write(answer)